In [56]:
from pyts.datasets import load_basic_motions
import numpy as np

In [57]:
bunch = load_basic_motions()
X_train, X_test, y_train, y_test = load_basic_motions(return_X_y=True)

In [58]:
X_train.shape

(40, 6, 100)

In [455]:
class reservoir_features:
    '''
    creates an object associated with a multivariate dataset
    
    
    '''
    def __init__(self,data,num_features):
        '''
        data: Takes in a multidimensional array (x * y * z) - z>y
        Initializes it
        x: Different timeseries
        y: attributes for a given timeseries observation
        z: timestamped observations (features)
        
        num_features: you must specify the dimension you want to reduce it to
        
        '''
        self.features = []
        self.filters_used = []
        self.original_data = data.copy()
        self.data = data.copy()
        self.num_features = num_features
        self.x = data.shape[0]
        self.y = data.shape[1]
        self.z = data.shape[2]
        # perform checks 
    
    def normalize(self):
        '''
        modifies the original data and performs a min max normalization
        '''
        for i in range(self.original_data.shape[0]):
            for j in range(self.original_data.shape[1]):
                self.data[i][j] = (self.original_data[i][j] - self.original_data[i][j].min())/(self.original_data[i][j].max()-self.original_data[i][j].min())
    
    
    def filters(self,stride_len = 1, num_filters = 1):
        '''
        stride_len: num of columns to skip after each filter multiplication
        num_filters: you can specify the number of filters you need; each filter will be of a differnt size
        size of filter = n*m (n = # of rows, m = # of columns)
        '''
        n = self.y
        vals = self._get_m_padding(stride_len)
        
        #Edge case vals is empty/smaller than num_filters
        
        for iteration in range(num_filters):
            m, pad_size = vals[iteration]
            temp_data = self._create_padding(pad_size)
            
            filter_a = np.random.random((n,m))
            print("filter of size ", str(n), "*", str(m), "was created\n")
            print("Right padding of ", str(pad_size), "was used\n\n")
            self.filters_used.append(filter_a)
            
            temp_features =[]
            for i in range(self.x):
                temp = []
                j = 0
                while j + m < temp_data.shape[2]:
                    temp.append((filter_a*temp_data[i,:,j:j+m]).mean())
                    j+=stride_len
                temp_features.append(temp)
            self.features.append(temp_features)
    
    
    def _get_m_padding(self,stride_len):
        '''
        stride_len: 
        based on stride length,& num_features, we calculate possible filter size and padding
        padding is only right padded and is controlled to be between 0 and 20
        '''
        vals =[]
        for padding in range(0,20):
            m = self.z + padding-(self.num_features)*stride_len 
            if abs(m)==m:
                vals.append([m,padding])
        return vals
    
    def _create_padding(self,pad_size):
        '''
        based on pad_size, add right padding
        '''
        return np.pad(self.data,((0,0),(0,0),(0,int(pad_size))),'constant')
    
    
    def result_features(self):
        '''
        if multiple filters were added, takes the average result
        '''
        ans =[]
        for timeseries in range(len(self.features[0])):
            temp =[]
            for feature in range(len(self.features[0][0])):
                val = np.mean([self.features[filter][timeseries][feature] for filter in range(len(self.features))])
                temp.append(val)
            ans.append(temp)
        return ans
        

In [456]:
#create object
a = reservoir_features(X_train,num_features = 16)

In [457]:
#normalize
a.normalize()

In [458]:
#create 2 filters
a.filters(stride_len = 1, num_filters = 2)

filter of size  6 * 84 was created

Right padding of  0 was used


filter of size  6 * 85 was created

Right padding of  1 was used




In [459]:
len(a.features)

2

In [460]:
a.filters(stride_len = 3, num_filters = 1)

filter of size  6 * 52 was created

Right padding of  0 was used




In [461]:
len(a.features)

3

In [462]:
ans = a.result_features()

In [463]:
len(ans)

40

In [464]:
len(ans[0])

16

In [465]:
ans

[[0.24757958129900523,
  0.24546304127796623,
  0.24470653794527872,
  0.24427063086839665,
  0.24254363138752252,
  0.24169957684893498,
  0.23774997780755833,
  0.23742999645051763,
  0.23861181224725705,
  0.23856724215219557,
  0.23952010802532941,
  0.24054832544144636,
  0.24105075152934938,
  0.24036143861857653,
  0.2388035161680583,
  0.2384818811823448],
 [0.3374306791851711,
  0.33643823391575767,
  0.33900445633990106,
  0.3421238140566196,
  0.3420429890859111,
  0.3423324028471675,
  0.3425669887050269,
  0.34303647868640447,
  0.3429298513336195,
  0.3425664003872442,
  0.34240595990663164,
  0.3418942904097662,
  0.3417482374085168,
  0.34167872201892346,
  0.34167984941291846,
  0.3417357428977608],
 [0.27866256200055073,
  0.2784630627019065,
  0.2756543990376483,
  0.27171477296411756,
  0.2735852577314086,
  0.2733955896069942,
  0.27079097060736207,
  0.2659213288379891,
  0.2702908592151909,
  0.273368784932873,
  0.2712661976158682,
  0.26834181872138957,
  0.270